In [ ]:
#Import the following libraries
import pandas as pd
pd.set_option('display.max_columns',None)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

In [ ]:
# RECEIPTS
receipts_history = '1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE'
url='https://docs.google.com/spreadsheets/d/'+receipts_history+ '/export?format=xlsx'
receipts = pd.read_excel(url, sheet_name='Sheet1')
receipts.head(5)

,date,customer_id,receipt_amount,new_customer
0,2023-01-01,2653,67,1
1,2023-01-01,2731,271,1
2,2023-01-01,1277,231,0
3,2023-01-01,2094,107,0
4,2023-01-01,1314,416,0


In [ ]:
# PAYROLL
payroll_gg = '1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4'
url1='https://docs.google.com/spreadsheets/d/'+payroll_gg+ '/export?format=xlsx'
payroll = pd.read_excel(url1, sheet_name='Sheet1')
payroll.head(5)

,month,department,employee_name,position,paid
0,2023-01-01,Sales,John Doe,Sales Manager,1500
1,2023-01-01,Sales,Jane Smith,Sales Associate,600
2,2023-01-01,Sales,Jim Brown,Sales Associate,700
3,2023-01-01,Sales,Laura Miller,Sales Associate,800
4,2023-01-01,Marketing,Alice Johnson,Marketing Manager,1650


In [ ]:
# MONTHLY EXPENSE
monthly_exp = '10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw'
url2 = 'https://docs.google.com/spreadsheets/d/'+monthly_exp+ '/export?format=xlsx'
expense = pd.read_excel(url2, sheet_name='Sheet1')
expense.head(5)

,#,month,category,item,amount
0,1,2023-01-01,Server Costs,AWS Hosting,8000
1,2,2023-01-01,Server Costs,Google Cloud Storage,4000
2,3,2023-01-01,Software Licenses,Atlassian Jira,1200
3,4,2023-01-01,Software Licenses,Slack,800
4,5,2023-01-01,Software Licenses,Salesforce,1500


In [ ]:
# DAILY MARKETING SPENDING
daily_marketing_spending = '1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c'
url3 = 'https://docs.google.com/spreadsheets/d/'+daily_marketing_spending+ '/export?format=xlsx'
marketing = pd.read_excel(url3, sheet_name='Sheet1')
marketing.head(5)

,date,channel,spending
0,2023-01-01,Google Ads,784
1,2023-01-01,Facebook Ads,659
2,2023-01-01,LinkedIn Ads,729
3,2023-01-01,Twitter Ads,292
4,2023-01-02,Google Ads,935


In [ ]:
# CUSTOMER LIFESPAN
customer_lifespan = '1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI'
url4 = 'https://docs.google.com/spreadsheets/d/'+customer_lifespan+ '/export?format=xlsx'
customer = pd.read_excel(url4, sheet_name='Sheet1')
customer.columns = ['customer_id','start_date','churn_date']
customer["lifespan_days"] = (customer["churn_date"]-customer["start_date"]).dt.days
customer["lifespan_years"] = customer["lifespan_days"]/365
customer.head(5)

,customer_id,start_date,churn_date,lifespan_days,lifespan_years
0,1000,2021-11-15,2022-09-14,303,0.830137
1,1001,2022-04-15,2023-02-16,307,0.841096
2,1002,2022-10-30,2023-02-04,97,0.265753
3,1003,2021-08-22,2023-02-07,534,1.463014
4,1004,2021-08-23,2022-02-02,163,0.446575


In [ ]:
# tạo cột year cho các bảng
receipts['year'] = receipts['date'].dt.year
payroll['year'] = payroll['month'].dt.year
expense['year'] = expense['month'].dt.year
marketing['year'] = marketing['date'].dt.year

# check duplicate và lọc số khách hàng mới
receipts.groupby(['year','new_customer'])['customer_id'].nunique().reset_index()

,year,new_customer,customer_id
0,2023,0,600
1,2023,1,199


In [ ]:
# CAC = (total_sale + MKT ) / Number_of_new_customer
total_office_expense = expense[expense["year"]==2023]["amount"].sum()
total_payroll_expense = payroll[payroll["year"]==2023]["paid"].sum()
total_mkt_expense = marketing[marketing["year"]==2023]['spending'].sum()
total_new_customer = receipts[(receipts["year"]==2023)
    & (receipts["new_customer"]==1)]["customer_id"].nunique()
CAC = (total_office_expense+total_payroll_expense+total_mkt_expense)/total_new_customer
print("CAC = {:.2f}".format(CAC))

CAC = 1766.02


In [ ]:
# ARPU = Average Revenue Per User = TotalRevenue / Numberofuser
number_of_user = receipts["customer_id"].nunique()
total_revenue = receipts["receipt_amount"].sum()
ARPU = total_revenue/number_of_user
print("ARPU = {:.2f}".format(ARPU))

ARPU = 327.28


In [ ]:
# COGS: Cost of Goods Sold
COGS = total_office_expense + total_payroll_expense
print("COGS = {:.2f}".format(COGS))

COGS = 151960.00


In [ ]:
# Gross margin
GROSS_MARGIN = (total_revenue - COGS)/ total_revenue *100
print("Gross margin = {:.2f}".format(GROSS_MARGIN))

Gross margin = 37.00


In [ ]:
# LTV: Customer lifetime value =ARPU × Gross Margin × avg_lifespan
avg_lifespan = customer["lifespan_years"].mean()
LTV = ARPU * GROSS_MARGIN/100 * avg_lifespan
print("LTV = {:.2F}".format(LTV))

LTV = 97.95


In [ ]:
print("LTV/CAC = {:.2f}".format(LTV/CAC))

LTV/CAC = 0.06


**KẾT LUẬN:**
1.   CAC = 1766.02 - Chi phí để có 1 khách hàng mới là 1766.02$, do chưa đủ thông tin nên chưa thể so sánh CAC qua các năm để xem xét marketing hiệu quả hay không,

2.   ARPU = 327.28 - Doanh thu trung bình/ mỗi khách hàng là 327.28$

3.   COGS = 151960.00 - Giá vốn hàng bán (không bao gồm chi phí Marketing) là 151960$
4.   Gross margin = 37.00 - Biên lợi nhuận gộp => doanh nghiệp giữ 37% doanh thu sau khi trừ giá vốn hàng bán, đây là 1 mức trung bình, để nâng cao biên lợi nhuận gộp, doanh nghiệp cần giảm chi phí giá vốn hàng bán (COGS)

5.   LTV = 97.95 - Customer Lifetime Value: mỗi khách hàng mang lại khoảng 97.95$ trong suốt thời gian gắn bó với doanh nghiệp
6.   LTV/CAC = 0.06, chỉ số này < 1, tỷ lệ nhỏ  => doanh nghiệp chi nhiều tiền hơn để thu hút khách hàng so với số tiền kiếm được từ họ trong suốt vòng đời của khách hàng => Chứng minh đây là mô hình kinh doanh không bền vững.



